
# DM Project 3 — Regression (Starter Notebook)

This notebook scaffolds a clean, reproducible workflow for the Ames Housing regression task (Kaggle‑style).  
Update paths as needed to point to your local `dm_project_3/` folder that already contains `train.csv` and `test.csv`.

**Workflow overview**
1. Load data & sanity checks  
2. Minimal EDA (shapes, nulls, distributions)  
3. Preprocessing (numeric/ordinal/categorical) + feature engineering  
4. **Experiments** (at least three):  
   - **Exp A**: Baseline Linear Regression on engineered features  
   - **Exp B**: Regularized models (Ridge & Lasso) with CV  
   - **Exp C**: Gradient Boosting (e.g., HistGradientBoostingRegressor)  
5. Compare via cross‑validated RMSE (log‑target option included)  
6. Train best model on full train; predict `test.csv`; write `submission.csv`.


In [5]:

# If running in a fresh environment, you may need:
# pip install pandas numpy scikit-learn matplotlib

from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, make_scorer

import matplotlib.pyplot as plt

# Paths (edit if your notebook sits elsewhere)
DATA_DIR = Path('dm_project_3')
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH  = DATA_DIR / 'test.csv'

SEED = 42
np.random.seed(SEED)


In [6]:

train = pd.read_csv(train.csv)
test  = pd.read_csv(test.csv)

print(train.shape, test.shape)
display(train.head(3))
display(train.describe(include='all').T.head(15))


NameError: name 'train' is not defined

In [ ]:

# Basic nulls overview
nulls = train.isna().sum().sort_values(ascending=False)
display(nulls.head(20))

# Target
assert 'SalePrice' in train.columns, "train.csv must have 'SalePrice' column"
train['log_SalePrice'] = np.log1p(train['SalePrice'])  # option for log target

fig = plt.figure()
train['SalePrice'].hist(bins=50)
plt.title('SalePrice distribution')
plt.show()

fig = plt.figure()
train['log_SalePrice'].hist(bins=50)
plt.title('log1p(SalePrice) distribution')
plt.show()


In [ ]:

# Example feature engineering tailored to Ames:
# - TotalSF = GrLivArea + TotalBsmtSF
# - Age features: HouseAge, Remodeled (boolean), SinceRemodel
# - TotalBaths (Full + Half*0.5 + Basement baths)
# - QualityScore (e.g., OverallQual + KitchenQual/ExterQual ordinal maps)
#
# You can add/remove features in the 'featurize' function.

# Ordinal maps for selected quality columns (higher is better)
qual_map = {'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5}
exterqual_cols = ['ExterQual','KitchenQual','HeatingQC']

def featurize(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # Safe fills
    for c in ['TotalBsmtSF','GrLivArea','FullBath','HalfBath','BsmtFullBath','BsmtHalfBath']:
        if c in out.columns:
            out[c] = out[c].fillna(0)
    # TotalSF
    if set(['GrLivArea','TotalBsmtSF']).issubset(out.columns):
        out['TotalSF'] = out['GrLivArea'] + out['TotalBsmtSF']
    # Age features
    if 'YearBuilt' in out.columns and 'YrSold' in out.columns:
        out['HouseAge'] = out['YrSold'] - out['YearBuilt']
    if 'YearRemodAdd' in out.columns and 'YrSold' in out.columns:
        out['SinceRemodel'] = out['YrSold'] - out['YearRemodAdd']
        out['Remodeled'] = (out['YearRemodAdd'] != out['YearBuilt']).astype(int)
    # Baths
    for col in ['FullBath','HalfBath','BsmtFullBath','BsmtHalfBath']:
        if col not in out.columns:
            out[col] = 0
    out['TotalBaths'] = out['FullBath'] + 0.5*out['HalfBath'] +                         out['BsmtFullBath'] + 0.5*out['BsmtHalfBath']
    # QualityScore
    for c in exterqual_cols:
        if c in out.columns:
            out[c] = out[c].map(qual_map).fillna(0)
    if set(['OverallQual','KitchenQual','ExterQual']).issubset(out.columns):
        out['QualityScore'] = out['OverallQual'] + out['KitchenQual'] + out['ExterQual']
    return out

# Wrap for sklearn ColumnTransformer with FunctionTransformer
featurizer = FunctionTransformer(featurize, feature_names_out='one-to-one')


In [ ]:

# Identify column types
target = 'SalePrice'
drop_cols = [target] if target in train.columns else []
X_all = train.drop(columns=drop_cols, errors='ignore')
y_all = train[target]

# After featurize we will recompute columns, so we infer types on raw X
numeric_cols = [c for c in X_all.columns if pd.api.types.is_numeric_dtype(X_all[c])]
# Selected ordinal quality columns already mapped in featurize; treat as numeric afterwards

# Choose a small, interpretable categorical set for baseline
categorical_cols = [c for c in X_all.columns 
                    if (X_all[c].dtype == 'object') and c not in exterqual_cols]

numeric_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='median')),
    ('scale', StandardScaler(with_mean=False))  # sparse-safe
])

categorical_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)


In [ ]:

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(lambda yt, yp: rmse(yt, yp), greater_is_better=False)

# Toggle to run models on log-target; predictions will be expm1-transformed for metrics
USE_LOG_TARGET = True

def y_transform(y):
    return np.log1p(y) if USE_LOG_TARGET else y

def y_inverse(yhat):
    return np.expm1(yhat) if USE_LOG_TARGET else yhat

cv = KFold(n_splits=5, shuffle=True, random_state=SEED)


## Experiment A — Baseline OLS (LinearRegression)

In [ ]:

model_A = Pipeline(steps=[
    ('featurize', featurizer),
    ('preprocess', preprocess),
    ('reg', LinearRegression())
])

scores_A = []
for tr_idx, te_idx in cv.split(train):
    X_tr, X_te = train.iloc[tr_idx].copy(), train.iloc[te_idx].copy()
    y_tr, y_te = y_transform(y_all.iloc[tr_idx].values), y_all.iloc[te_idx].values

    model_A.fit(X_tr, y_tr)
    preds = y_inverse(model_A.predict(X_te))
    scores_A.append(rmse(y_te, preds))

print("Exp A (OLS) CV RMSE:", np.mean(scores_A), "+/-", np.std(scores_A))


## Experiment B — Ridge & Lasso (with simple alpha grid)

In [ ]:

alphas = [0.1, 0.5, 1.0, 3.0, 10.0, 30.0, 100.0]
results_B = []

for reg_name, reg in [('Ridge', Ridge), ('Lasso', Lasso)]:
    for a in alphas:
        pipe = Pipeline(steps=[
            ('featurize', featurizer),
            ('preprocess', preprocess),
            ('reg', reg(alpha=a, random_state=SEED) if reg_name=='Lasso' else reg(alpha=a))
        ])
        fold_scores = []
        for tr_idx, te_idx in cv.split(train):
            X_tr, X_te = train.iloc[tr_idx].copy(), train.iloc[te_idx].copy()
            y_tr, y_te = y_transform(y_all.iloc[tr_idx].values), y_all.iloc[te_idx].values
            pipe.fit(X_tr, y_tr)
            preds = y_inverse(pipe.predict(X_te))
            fold_scores.append(rmse(y_te, preds))
        results_B.append((reg_name, a, np.mean(fold_scores), np.std(fold_scores)))

results_B_df = pd.DataFrame(results_B, columns=['Model','alpha','CV_RMSE','CV_STD']).sort_values('CV_RMSE')
display(results_B_df.head(10))


## Experiment C — Tree‑based: HistGradientBoostingRegressor

In [ ]:

# Use numeric-encoded features only for HGB to keep things fast: use OHE too via preprocess
model_C = Pipeline(steps=[
    ('featurize', featurizer),
    ('preprocess', preprocess),
    ('reg', HistGradientBoostingRegressor(random_state=SEED))
])

scores_C = []
for tr_idx, te_idx in cv.split(train):
    X_tr, X_te = train.iloc[tr_idx].copy(), train.iloc[te_idx].copy()
    y_tr, y_te = y_transform(y_all.iloc[tr_idx].values), y_all.iloc[te_idx].values
    model_C.fit(X_tr, y_tr)
    preds = y_inverse(model_C.predict(X_te))
    scores_C.append(rmse(y_te, preds))

print("Exp C (HGB) CV RMSE:", np.mean(scores_C), "+/-", np.std(scores_C))


## Pick the best model (manually for now) and refit on full train

In [ ]:

# Example: choose the best mean CV RMSE from the experiments above.
# For reproducibility, you can paste the best params here.
best_model = model_C  # <-- change to model_A or a specific Ridge/Lasso from Experiment B

# Fit on FULL training data
best_model.fit(train, y_transform(y_all.values))

# Create Kaggle submission on test.csv
test_pred = y_inverse(best_model.predict(test))
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': test_pred})
sub_path = DATA_DIR / 'submission.csv'
sub.to_csv(sub_path, index=False)
sub_path


## Optional: Local train/validation split (skip Kaggle)

In [ ]:

# If you don't want to submit to Kaggle, you can evaluate with a final holdout:
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(train, y_all, test_size=0.2, random_state=SEED)
best_model.fit(X_tr, y_transform(y_tr.values))
preds = y_inverse(best_model.predict(X_te))
print("Holdout RMSE:", rmse(y_te.values, preds))



### Notes & Next Steps
- Try more engineered features (e.g., `TotalPorchSF`, `IsNew`, neighborhood medians, interaction terms).
- Replace `HistGradientBoostingRegressor` with `XGBRegressor` or `LGBMRegressor` if allowed in your environment.
- Tune hyperparameters using `GridSearchCV` or `RandomizedSearchCV`.
- Document each experiment: what changed, why, and the results.
